<a href="https://colab.research.google.com/github/dantae74/machine-learning/blob/main/stock-etc/99-rnn-lotto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

모두를 위한 머신러닝에서 가져왔습니다.
# RNN-lotto

In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from io import BytesIO
import requests

In [4]:
noEnd ='965' # 마지막 로또 회차 번호
url = 'https://dhlottery.co.kr/gameResult.do?method=allWinExel&gubun=byWin&nowPage=&drwNoStart=1&drwNoEnd={}'.format(noEnd)

# user-agent
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"

In [5]:
result = requests.get(url, headers={"user-agent":user_agent})

In [6]:
result

<Response [200]>

In [7]:
f = BytesIO(result.content)
# df = pd.read_excel(f,sheet_name='excel',skiprows=3)
html_result = pd.read_html(f)

In [25]:
df = pd.DataFrame(html_result[1])
df = df.drop([0])
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
1,년도,회차,추첨일,당첨자수,당첨금액,당첨자수,당첨금액,당첨자수,당첨금액,당첨자수,당첨금액,당첨자수,당첨금액,1,2,3,4,5,6,보너스
2,2021,965,2021.05.29,7,"3,403,348,929원",46,"86,316,821원",2389,"1,662,024원",122129,"50,000원",2043257,"5,000원",2,13,25,28,29,36,34
3,2021,964,2021.05.22,10,"2,345,861,063원",52,"75,187,855원",2447,"1,597,781원",123780,"50,000원",2061560,"5,000원",6,21,36,38,39,43,30
4,2021,963,2021.05.15,15,"1,476,478,125원",68,"54,282,285원",3137,"1,176,665원",141987,"50,000원",2219439,"5,000원",6,12,19,23,34,42,35
5,2021,962,2021.05.08,12,"1,940,906,094원",73,"53,175,510원",2581,"1,503,996원",126528,"50,000원",2119457,"5,000원",1,18,28,31,34,43,40


In [9]:
df.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
            19],
           dtype='int64')

In [45]:
columns = [col for i, col in enumerate(df.iloc[0])]
columns

['년도',
 '회차',
 '추첨일',
 '당첨자수',
 '당첨금액',
 '당첨자수',
 '당첨금액',
 '당첨자수',
 '당첨금액',
 '당첨자수',
 '당첨금액',
 '당첨자수',
 '당첨금액',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '보너스']

In [46]:
df.columns = columns

In [55]:
df = df.drop([1])

In [57]:
df = df.sort_values('회차', ascending=True)

,년도,회차,추첨일,당첨자수,당첨금액,당첨자수,당첨금액,당첨자수,당첨금액,당첨자수,당첨금액,당첨자수,당첨금액,1,2,3,4,5,6,보너스
966,2002,1,2002.12.07,0,0원,1,"143,934,100원",28,"5,140,500원",2537,"113,400원",40155,"10,000원",10,23,29,33,37,40,16
957,2003,10,2003.02.08,13,"6,430,437,900원",236,"40,813,400원",11247,"856,400원",703234,"27,300원",3410846,"10,000원",9,25,30,33,41,44,6
867,2004,100,2004.10.30,4,"3,315,315,525원",36,"61,394,732원",1544,"1,431,484원",76899,"57,484원",1235777,"5,000원",1,7,11,23,37,42,6
866,2004,101,2004.11.06,5,"2,707,297,500원",38,"59,370,560원",1405,"1,605,752원",73210,"61,634원",1246344,"5,000원",1,3,17,32,35,45,8
865,2004,102,2004.11.13,9,"1,457,153,067원",24,"91,072,067원",1352,"1,616,664원",75476,"57,919원",1322337,"5,000원",17,22,24,26,35,40,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,2021,964,2021.05.22,10,"2,345,861,063원",52,"75,187,855원",2447,"1,597,781원",123780,"50,000원",2061560,"5,000원",6,21,36,38,39,43,30
2,2021,965,2021.05.29,7,"3,403,348,929원",46,"86,316,821원",2389,"1,662,024원",122129,"50,000원",2043257,"5,000원",2,13,25,28,29,36,34
870,2004,97,2004.10.09,9,"1,496,214,000원",46,"48,789,587원",1543,"1,454,518원",77227,"58,123원",1282678,"5,000원",6,7,14,15,20,36,3
869,2004,98,2004.10.16,4,"3,177,972,300원",45,"47,081,072원",1880,"1,126,941원",87818,"48,251원",1385812,"5,000원",6,9,16,23,24,32,43


In [58]:
df[['1','2','3','4','5','6']]

,1,2,3,4,5,6
2,2,13,25,28,29,36
3,6,21,36,38,39,43
4,6,12,19,23,34,42
5,1,18,28,31,34,43
6,11,20,29,31,33,42
...,...,...,...,...,...,...
962,16,24,29,40,41,42
963,14,27,30,31,40,42
964,11,16,19,21,27,31
965,9,13,21,25,32,42


In [59]:
ncount = np.zeros((1,45), dtype=float)

In [61]:
for i in range(df.shape[0]):
  for j in range(6):
    col = int(df.iloc[i][str(j+1)])
    ncount[0][col-1] += 1.0

  if i == 0:
    X = ncount
  else:
    X = np.append(X, ncount, axis=0)

print('x.shape:', X.shape)

x.shape: (965, 45)


In [66]:
df_y = df[['1','2','3','4','5','6']]
Y = df_y.values
print('x.shape:', X.shape)
print('y.shape:', Y.shape)
Y

x.shape: (965, 45)
y.shape: (965, 6)


array([['2', '13', '25', '28', '29', '36'],
       ['6', '21', '36', '38', '39', '43'],
       ['6', '12', '19', '23', '34', '42'],
       ...,
       ['11', '16', '19', '21', '27', '31'],
       ['9', '13', '21', '25', '32', '42'],
       ['10', '23', '29', '33', '37', '40']], dtype=object)

In [63]:
y_onehot = []
tmponehot = np.zeros(45).tolist()
tmponehot[3] = 1.0
y_onehot.append(tmponehot)
y_onehot.append(tmponehot)

np.array(y_onehot).shape

(2, 45)

In [64]:
def make_onehot(value):
  onehot = np.zeros(45).tolist()
  onehot[value-1] = 1.0
  return onehot

In [68]:
y_onehot = []
for i in range(Y.shape[0]):
  for j in range(6):
    y_onehot.append(make_onehot(int(Y[i][j])))

y_onehot = np.array(y_onehot)
y_onehot = np.reshape(y_onehot, (Y.shape[0],6,45))
print('y_onehot.shape:', y_onehot.shape)

y_onehot.shape: (965, 6, 45)


In [69]:
X.shape

(965, 45)

In [70]:
def build_dataset(x_time_series, y_time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(y_time_series) - seq_length):
        x = x_time_series[i:i + seq_length, :]
        y = y_time_series[i + seq_length - 1, :]
        dataX.append(x)
        dataY.append(y)
    return np.array(dataX), np.array(dataY)

In [71]:
x_data, y_data = build_dataset(X, y_onehot, 6)

In [72]:
print('x_data.shape:', x_data.shape)
print('y_data.shape:', y_data.shape)

x_data.shape: (959, 6, 45)
y_data.shape: (959, 6, 45)


In [73]:
model = keras.Sequential();
model.add(keras.layers.LSTM(units=45, input_shape=(6, 45), return_sequences=True))
model.add(keras.layers.LSTM(units=45, return_sequences=True))
model.add(keras.layers.Dense(units=45, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 6, 45)             16380     
_________________________________________________________________
lstm_1 (LSTM)                (None, 6, 45)             16380     
_________________________________________________________________
dense (Dense)                (None, 6, 45)             2070      
Total params: 34,830
Trainable params: 34,830
Non-trainable params: 0
_________________________________________________________________


In [74]:
model.fit(x_data, y_data, epochs=1200, verbose=0)

In [75]:
x_test = x_data[-1,:]
x_test = np.delete(x_test, 0, axis=0)
x_test = np.append(x_test, X[-1:], axis=0)
x_test = x_test.reshape((1,6,45))
print('x_test.shape:', x_test.shape)

x_test.shape: (1, 6, 45)


In [76]:
pred_y = model.predict(x_test)

In [77]:
for i in range(6):
  print(np.argmax(pred_y[0][i]) + 1)

1
10
19
26
34
45
